# Загрузка необходимых библиотек

In [15]:
pip install optuna

Note: you may need to restart the kernel to use updated packages.


In [16]:
import gc
gc.collect()

0

In [17]:
import psutil
print(f"Доступно RAM: {psutil.virtual_memory().available / (1024 ** 3):.2f} GB")

Доступно RAM: 0.81 GB


In [18]:
!pip install xgboost

In [19]:
from xgboost import XGBClassifier

In [20]:
from sklearn.metrics import classification_report
from sklearn.ensemble import BaggingClassifier

In [21]:
import pandas as pd
import numpy as np
from pycaret.regression import *
from sklearn.model_selection import train_test_split, KFold
from sklearn.model_selection import train_test_split, KFold, StratifiedKFold
from collections import Counter
from imblearn.over_sampling import SMOTE
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint, uniform
import optuna
from sklearn.model_selection import cross_val_score
import matplotlib.pyplot as plt
from sklearn.tree import plot_tree
from sklearn.tree import export_text
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, mean_absolute_percentage_error

In [22]:
from pycaret.classification import *

In [23]:
!pip install catboost

# Загрузка и подготовка данных. 
- В соответствии с индивидуальным вариантом загрузите предобработан
ные датасеты в формате CSV для решения задачи регрессии и классификации. 
- При разделении данных на обучающую и тестовую выборки используйте 
два метода с соответствующими параметрами: hold-out и k-fold.

In [24]:
kredit_data = pd.read_csv('data_filter3.csv')
kredit_data = kredit_data.drop(columns=["protection", "unprotected_online_order"])

In [25]:
house_data = pd.read_csv('data_new.csv')
district_rank = {
    "Federal Way": 1,      #самый дешёвый (289384.88)
    "Auburn": 2,
    "Kent": 3,
    "Enumclaw": 4,
    "Maple Valley": 5,
    "Renton": 6,
    "Black Diamond": 7,
    "Duvall": 8,
    "North Bend": 9,
    "Carnation": 10,
    "Kenmore": 11,
    "Vashon": 12,
    "Bothell": 13,
    "Snoqualmie": 14,
    "Seattle": 15,
    "Fall City": 16,
    "Issaquah": 17,
    "Woodinville": 18,
    "Kirkland": 19,
    "Redmond": 20,
    "Sammamish": 21,
    "Bellevue": 22,
    "Mercer Island": 23,
    "Medina": 24           #самый дорогой (2160606.60)
}

house_data['district_code'] = house_data['district'].map(district_rank)
house_data.drop('district', axis=1, inplace=True)
house_data['district_code'] = house_data['district_code'].astype(int)

In [26]:
kredit_data.head()

,distance_from_home,distance_from_last_transaction,ratio_to_median_purchase_price,repeat_retailer,used_chip,used_pin_number,online_order,fraud
0,57.877857,0.311140,1.945940,1,1,0,0,0
1,10.829943,0.175592,1.294219,1,0,0,0,0
2,5.091079,0.805153,0.427715,1,0,0,1,0
3,2.247564,5.600044,0.362663,1,1,0,1,0
4,44.190936,0.566486,2.222767,1,1,0,1,0


In [27]:
data_new=kredit_data.copy()
data_new.to_csv("data_newKredit.csv", index=False)

In [28]:
house_data.head()

,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,...,long,sqft_living15,sqft_lot15,year_sale,age,age_renovated,lat_center,long_center,distance_to_center,district_code
0,221900,3,1.00,1180,5650,1.0,0,0,3,7,...,-122.257,1340,5650,2014,60,60,47.6062,-122.3321,11.98,15
1,538000,3,2.25,2570,7242,2.0,0,0,3,7,...,-122.319,1690,7639,2014,64,24,47.6062,-122.3321,12.80,15
2,180000,2,1.00,770,10000,1.0,0,0,3,6,...,-122.233,2720,8062,2015,83,83,47.7573,-122.2440,2.31,11
3,604000,4,3.00,1960,5000,1.0,0,0,5,7,...,-122.393,1360,5000,2014,50,50,47.6062,-122.3321,10.54,15
4,510000,3,2.00,1680,8080,1.0,0,0,3,8,...,-122.045,1800,7503,2015,29,29,47.6163,-122.0356,0.71,21


## Разделение данных на обучающую и тестовые выборки

### KreditCard (классификация)

In [29]:
X_clf = kredit_data.drop('fraud', axis=1)
y_clf = kredit_data['fraud']
print("Исходное распределение классов:", Counter(y_clf))

Исходное распределение классов: Counter({0: 912597, 1: 87403})


In [30]:
# Hold-out разделение (20% тест)
X_train_clf, X_test_clf, y_train_clf, y_test_clf = train_test_split(
    X_clf, y_clf,
    test_size=0.2,          # 20% в тест
    random_state=42,
    stratify=y_clf
)

In [31]:
print("Train распределение:", Counter(y_train_clf))
print("Test распределение:", Counter(y_test_clf))

Train распределение: Counter({0: 730078, 1: 69922})
Test распределение: Counter({0: 182519, 1: 17481})


In [32]:
X_train_clf = X_train_clf.reset_index(drop=True)
y_train_clf = y_train_clf.reset_index(drop=True)

In [33]:
print(Counter(y_train_clf))
print(Counter(X_train_clf))

Counter({0: 730078, 1: 69922})
Counter({'distance_from_home': 1, 'distance_from_last_transaction': 1, 'ratio_to_median_purchase_price': 1, 'repeat_retailer': 1, 'used_chip': 1, 'used_pin_number': 1, 'online_order': 1})


In [ ]:
# KFold
clf = RandomForestClassifier(
    n_estimators=200,
    max_depth=10,
    min_samples_leaf=5,
    class_weight='balanced',
    random_state=42,
    n_jobs=-1
)

# Для хранения метрик
accuracy_scores = []
precision_scores = []
recall_scores = []
f1_scores = []

skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

for fold, (train_idx, val_idx) in enumerate(skf.split(X_train_clf, y_train_clf)):
    print(f"\nFold {fold + 1}:")

    # 1. Разделение
    X_fold_train = X_train_clf.iloc[train_idx]
    y_fold_train = y_train_clf.iloc[train_idx]
    X_fold_val = X_train_clf.iloc[val_idx]
    y_fold_val = y_train_clf.iloc[val_idx]

    # 2. Балансировка
    smote = SMOTE(sampling_strategy=1.0, random_state=42)
    X_balanced, y_balanced = smote.fit_resample(X_fold_train, y_fold_train)

    print("После балансировки:", Counter(y_balanced))

    # 3. Обучение модели
    clf.fit(X_balanced, y_balanced)

    # 4. Предсказание и метрики
    y_pred = clf.predict(X_fold_val)
    acc = accuracy_score(y_fold_val, y_pred)
    prec = precision_score(y_fold_val, y_pred)
    rec = recall_score(y_fold_val, y_pred)
    f1 = f1_score(y_fold_val, y_pred)

    print(f"Accuracy: {acc:.4f}, Precision: {prec:.4f}, Recall: {rec:.4f}, F1: {f1:.4f}")

    accuracy_scores.append(acc)
    precision_scores.append(prec)
    recall_scores.append(rec)
    f1_scores.append(f1)

# 5. Средние метрики по всем фолдам
print("\nСредние метрики по кросс-валидации:")
print(f"Accuracy: {np.mean(accuracy_scores):.4f}")
print(f"Precision: {np.mean(precision_scores):.4f}")
print(f"Recall: {np.mean(recall_scores):.4f}")
print(f"F1: {np.mean(f1_scores):.4f}")


Fold 1:
После балансировки: Counter({0: 584062, 1: 584062})


### House (регрессия)

In [ ]:
X_reg = house_data.drop("price", axis=1)
y_reg = house_data["price"]

In [ ]:
# Hold-out
X_train_reg, X_test_reg, y_train_reg, y_test_reg = train_test_split(
    X_reg, y_reg,
    test_size=0.2,       # 20% в тест
    random_state=42
)

In [ ]:
kf = KFold(n_splits=5, shuffle=True, random_state=42)
reg = RandomForestRegressor(
    n_estimators=200,
    max_depth=15,
    min_samples_leaf=4,
    random_state=42,
    n_jobs=-1
)

mse_scores = []
mae_scores = []
r2_scores = []

for fold, (train_idx, val_idx) in enumerate(kf.split(X_train_reg)):
    print(f"\nFold {fold + 1}:")

    # Разделение данных
    X_fold_train, X_fold_val = X_train_reg.iloc[train_idx], X_train_reg.iloc[val_idx]
    y_fold_train, y_fold_val = y_train_reg.iloc[train_idx], y_train_reg.iloc[val_idx]

    # Обучение модели
    reg.fit(X_fold_train, y_fold_train)

    # Предсказания
    y_pred = reg.predict(X_fold_val)

    # Метрики
    mse = mean_squared_error(y_fold_val, y_pred)
    mae = mean_absolute_error(y_fold_val, y_pred)
    r2 = r2_score(y_fold_val, y_pred)

    print(f"MSE: {mse:.2f}, MAE: {mae:.2f}, R²: {r2:.4f}")

    # Сохраняем метрики
    mse_scores.append(mse)
    mae_scores.append(mae)
    r2_scores.append(r2)

# Усреднённые метрики
print("\nСредние метрики по кросс-валидации:")
print(f"MSE: {np.mean(mse_scores):.2f}")
print(f"MAE: {np.mean(mae_scores):.2f}")
print(f"R²: {np.mean(r2_scores):.4f}")

### Что используем в работе?

Классификация:
- X_train_clf, y_train_clf  (Основной тренировочный набор)
- X_test_clf, y_test_clf  (Финальный тестовый набор)

Регрессия:
- X_train_reg, y_train_reg  (Основной тренировочный набор)
- X_test_reg, y_test_reg  (Финальный тестовый набор)

# Решение задачи регрессии и классификации

##  Деревья решений: DecisionTreeRegressor и DecisionTreeClassifier

Решите задачу регрессии и классификации на ваших данных с использованием деревьев решений: DecisionTreeRegressor и DecisionTreeClassifier из 
Scikit-learn

### DecisionTreeClassifier

In [ ]:
X_small, _, y_small, _ = train_test_split(
    X_train_clf, y_train_clf,
    train_size=0.01,
    stratify=y_train_clf,
    random_state=42
)

In [ ]:
param_grid = {
    'criterion': ['gini', 'entropy', 'log_loss'],
    'max_depth': [5, 10, 15, None],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 5],
    'max_features': ['sqrt', 'log2', None],
    'max_leaf_nodes': [None, 10, 20],
    'class_weight': [None, 'balanced']
}
grid_clf_dtc = GridSearchCV(DecisionTreeClassifier(random_state=42), param_grid,cv=3, scoring='f1', n_jobs=-1)
grid_clf_dtc.fit(X_small, y_small)
best_grid_params_clf_dtc = grid_clf_dtc.best_params_
print("Лучшие параметры от GridSearch:", best_grid_params_clf_dtc)

In [ ]:
param_dist = {
    'criterion': ['gini', 'entropy', 'log_loss'],
    'max_depth': randint(3, 20),
    'min_samples_split': randint(2, 20),
    'min_samples_leaf': randint(1, 10),
    'max_features': ['sqrt', 'log2', None],
    'max_leaf_nodes': [None, 10, 20, 30],
    'class_weight': [None, 'balanced']
}
random_search_clf_dtc = RandomizedSearchCV(DecisionTreeClassifier(random_state=42),
                                   param_distributions=param_dist,
                                   n_iter=30,
                                   cv=3,
                                   scoring='f1',
                                   random_state=42,
                                   n_jobs=-1)
random_search_clf_dtc.fit(X_small, y_small)
best_random_params_clf_dtc = random_search_clf_dtc.best_params_
print("Лучшие параметры от RandomizedSearch:", best_random_params_clf_dtc)

In [ ]:
def objective(trial):
    clf = DecisionTreeClassifier(
        criterion=trial.suggest_categorical('criterion', ['gini', 'entropy', 'log_loss']),
        max_depth=trial.suggest_int('max_depth', 3, 20),
        min_samples_split=trial.suggest_int('min_samples_split', 2, 20),
        min_samples_leaf=trial.suggest_int('min_samples_leaf', 1, 10),
        max_features=trial.suggest_categorical('max_features', ['sqrt', 'log2', None]),
        max_leaf_nodes=trial.suggest_categorical('max_leaf_nodes', [None, 10, 20, 30]),
        class_weight=trial.suggest_categorical('class_weight', [None, 'balanced']),
        random_state=42
    )
    score = cross_val_score(clf, X_small, y_small, cv=3, scoring='f1').mean()
    return score

study_clf_dtc = optuna.create_study(direction='maximize')
study_clf_dtc.optimize(objective, n_trials=30)
best_optuna_params_clf_dtc = study_clf_dtc.best_params
print("Лучшие параметры от Optuna:", best_optuna_params_clf_dtc)

In [ ]:
# Обучение моделей на полном тренировочном наборе
grid_clf = DecisionTreeClassifier(**best_grid_params_clf_dtc, random_state=42)
grid_clf.fit(X_train_clf, y_train_clf)

rand_clf = DecisionTreeClassifier(**best_random_params_clf_dtc, random_state=42)
rand_clf.fit(X_train_clf, y_train_clf)

optuna_clf = DecisionTreeClassifier(**best_optuna_params_clf_dtc, random_state=42)
optuna_clf.fit(X_train_clf, y_train_clf)

In [ ]:
# Предсказания на тестовой выборке
y_pred_grid = grid_clf.predict(X_test_clf)
y_pred_rand = rand_clf.predict(X_test_clf)
y_pred_optuna = optuna_clf.predict(X_test_clf)

# Сравнение метрик в таблице
comparison_clf = pd.DataFrame({
    "Метрика": ["Accuracy", "Precision", "Recall", "F1-score"],
    "GridSearch": [
        accuracy_score(y_test_clf, y_pred_grid),
        precision_score(y_test_clf, y_pred_grid),
        recall_score(y_test_clf, y_pred_grid),
        f1_score(y_test_clf, y_pred_grid)
    ],
    "RandomSearch": [
        accuracy_score(y_test_clf, y_pred_rand),
        precision_score(y_test_clf, y_pred_rand),
        recall_score(y_test_clf, y_pred_rand),
        f1_score(y_test_clf, y_pred_rand)
    ],
    "Optuna": [
        accuracy_score(y_test_clf, y_pred_optuna),
        precision_score(y_test_clf, y_pred_optuna),
        recall_score(y_test_clf, y_pred_optuna),
        f1_score(y_test_clf, y_pred_optuna)
    ]
})

print("Сравнение методов подбора для классификации:")
print(comparison_clf)

In [ ]:
final_params = best_optuna_params_clf_dtc
final_dtc_clf = DecisionTreeClassifier(**final_params, random_state=42)
final_dtc_clf.fit(X_train_clf, y_train_clf)

In [ ]:
y_pred_clf = final_dtc_clf.predict(X_test_clf)

clf_metrics = {
    "Метрика": ["Accuracy", "Precision", "Recall", "F1-score"],
    "Значение": [
        accuracy_score(y_test_clf, y_pred_clf),
        precision_score(y_test_clf, y_pred_clf),
        recall_score(y_test_clf, y_pred_clf),
        f1_score(y_test_clf, y_pred_clf)
    ]
}

clf_metrics__dtc_df = pd.DataFrame(clf_metrics)
print("Метрики для классификации:")
print(clf_metrics__dtc_df)

In [ ]:
plt.figure(figsize=(50, 12))
plot_tree(final_dtc_clf,
          feature_names=X_train_clf.columns,
          class_names=['0', '1'],
          filled=True,
          rounded=True,
          fontsize=10)
plt.title("Дерево решений — Классификация")
plt.show()

In [ ]:
# Выводим правила в виде текста
rules_text = export_text(final_dtc_clf, feature_names=list(X_train_clf.columns))
print("Решающие правила дерева (в текстовом виде):\n")
print(rules_text)

### DecisionTreeRegressor

In [ ]:
X_small_reg, _, y_small_reg, _ = train_test_split(
    X_train_reg, y_train_reg,
    train_size=0.01,
    random_state=42
)

In [ ]:
from sklearn.tree import DecisionTreeRegressor
param_grid_reg = {
    'criterion': ['squared_error', 'friedman_mse', 'absolute_error'],
    'max_depth': [5, 10, 15, None],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 5],
    'max_features': ['sqrt', 'log2', None],
    'max_leaf_nodes': [None, 10, 20]
}

grid_clt_reg = GridSearchCV(DecisionTreeRegressor(random_state=42),
                        param_grid=param_grid_reg,
                        cv=3,
                        scoring='neg_mean_squared_error',
                        n_jobs=-1)
grid_clt_reg.fit(X_small_reg, y_small_reg)
best_grid_params_clt_reg = grid_clt_reg.best_params_
print("Лучшие параметры от GridSearch:", best_grid_params_clt_reg)

In [ ]:
param_dist_reg = {
    'criterion': ['squared_error', 'friedman_mse', 'absolute_error'],
    'max_depth': randint(3, 20),
    'min_samples_split': randint(2, 20),
    'min_samples_leaf': randint(1, 10),
    'max_features': ['sqrt', 'log2', None],
    'max_leaf_nodes': [None, 10, 20, 30]
}

rand_clt_reg = RandomizedSearchCV(DecisionTreeRegressor(random_state=42),
                              param_distributions=param_dist_reg,
                              n_iter=30,
                              scoring='neg_mean_squared_error',
                              cv=3,
                              random_state=42,
                              n_jobs=-1)
rand_clt_reg.fit(X_small_reg, y_small_reg)
best_rand_params_clt_reg = rand_clt_reg.best_params_
print("Лучшие параметры от RandomizedSearch:", best_rand_params_clt_reg)

In [ ]:
def objective_reg(trial):
    model = DecisionTreeRegressor(
        criterion=trial.suggest_categorical('criterion', ['squared_error', 'friedman_mse', 'absolute_error']),
        max_depth=trial.suggest_int('max_depth', 3, 20),
        min_samples_split=trial.suggest_int('min_samples_split', 2, 20),
        min_samples_leaf=trial.suggest_int('min_samples_leaf', 1, 10),
        max_features=trial.suggest_categorical('max_features', ['sqrt', 'log2', None]),
        max_leaf_nodes=trial.suggest_categorical('max_leaf_nodes', [None, 10, 20, 30]),
        random_state=42
    )
    score = cross_val_score(model, X_small_reg, y_small_reg, cv=3, scoring='neg_mean_squared_error').mean()
    return score

study_clt_reg = optuna.create_study(direction='maximize')
study_clt_reg.optimize(objective_reg, n_trials=30)
best_optuna_params_clt_reg = study_clt_reg.best_params
print("Лучшие параметры от Optuna:", best_optuna_params_clt_reg)

In [ ]:
# Обучение моделей на полном тренировочном наборе
grid_reg = DecisionTreeRegressor(**best_grid_params_clt_reg, random_state=42)
grid_reg.fit(X_train_reg, y_train_reg)

rand_reg = DecisionTreeRegressor(**best_rand_params_clt_reg, random_state=42)
rand_reg.fit(X_train_reg, y_train_reg)

optuna_reg = DecisionTreeRegressor(**best_optuna_params_clt_reg, random_state=42)
optuna_reg.fit(X_train_reg, y_train_reg)

In [ ]:
# Предсказания на тесте
y_pred_grid_reg = grid_reg.predict(X_test_reg)
y_pred_rand_reg = rand_reg.predict(X_test_reg)
y_pred_optuna_reg = optuna_reg.predict(X_test_reg)

# Метрики
comparison_reg = pd.DataFrame({
    "Метрика": ["MSE", "MAE", "R2", "MAPE"],
    "GridSearch": [
        mean_squared_error(y_test_reg, y_pred_grid_reg),
        mean_absolute_error(y_test_reg, y_pred_grid_reg),
        r2_score(y_test_reg, y_pred_grid_reg),
        mean_absolute_percentage_error(y_test_reg, y_pred_grid_reg)
    ],
    "RandomSearch": [
        mean_squared_error(y_test_reg, y_pred_rand_reg),
        mean_absolute_error(y_test_reg, y_pred_rand_reg),
        r2_score(y_test_reg, y_pred_rand_reg),
        mean_absolute_percentage_error(y_test_reg, y_pred_rand_reg)
    ],
    "Optuna": [
        mean_squared_error(y_test_reg, y_pred_optuna_reg),
        mean_absolute_error(y_test_reg, y_pred_optuna_reg),
        r2_score(y_test_reg, y_pred_optuna_reg),
        mean_absolute_percentage_error(y_test_reg, y_pred_optuna_reg)
    ]
})

# Устанавливаем формат отображения чисел без экспонент
pd.options.display.float_format = '{:,.4f}'.format

# Выводим таблицу с форматированием
print("\nСравнение методов подбора для регрессии:")
print(comparison_reg)

In [ ]:
final_params_reg = best_grid_params_clt_reg
final_reg_clt = DecisionTreeRegressor(**final_params_reg, random_state=42)
final_reg_clt.fit(X_train_reg, y_train_reg)

In [ ]:
y_pred_reg = final_reg_clt.predict(X_test_reg)

reg_metrics = {
    "Метрика": ["MSE", "MAE", "R2", "MAPE"],
    "Значение": [
        mean_squared_error(y_test_reg, y_pred_reg),
        mean_absolute_error(y_test_reg, y_pred_reg),
        r2_score(y_test_reg, y_pred_reg),
        mean_absolute_percentage_error(y_test_reg, y_pred_reg)
    ]
}

reg_metrics_clt_df = pd.DataFrame(reg_metrics)
print("\nМетрики для регрессии:")
print(reg_metrics_clt_df)

In [ ]:
plt.figure(figsize=(45, 12))
plot_tree(final_reg_clt,
          feature_names=X_train_reg.columns,
          filled=True,
          rounded=True,
          fontsize=10)
plt.title("Дерево решений — Регрессия")
plt.show()

In [ ]:
rules_text_reg = export_text(final_reg_clt, feature_names=list(X_train_reg.columns))
print("Решающие правила дерева (регрессия):\n")
print(rules_text_reg)

## Ансамбли из Scikit-learn: BaggingRegressor, BaggingClassifier

Решите задачу регрессии и классификации на ваших данных с использованием ансамблей из Scikit-learn: 
- BaggingRegressor
- BaggingClassifier 

### BaggingClassifier

In [ ]:
X_small_bag_clf, _, y_small_bag_clf, _ = train_test_split(X_train_clf, y_train_clf, 
                                                          train_size=0.01, 
                                                          stratify=y_train_clf, 
                                                          random_state=42)

In [ ]:
param_grid_bag_clf = {
    'n_estimators': [10, 30, 50],
    'max_samples': [0.5, 0.7, 1.0],
    'max_features': [0.5, 0.7, 1.0],
    'bootstrap': [True, False]
}

grid_bag_clf = GridSearchCV(BaggingClassifier(estimator=RandomForestClassifier(), random_state=42),
                            param_grid_bag_clf,
                            cv=3,
                            scoring='f1',
                            n_jobs=-1)

grid_bag_clf.fit(X_small_bag_clf, y_small_bag_clf)
best_params_grid_bag_clf = grid_bag_clf.best_params_
print("Лучшие параметры от GridSearch:", best_params_grid_bag_clf)

In [ ]:
param_dist_bag_clf = {
    'n_estimators': randint(10, 100),
    'max_samples': uniform(0.5, 0.5),
    'max_features': uniform(0.5, 0.5),
    'bootstrap': [True, False]
}

random_bag_clf = RandomizedSearchCV(BaggingClassifier(estimator=RandomForestClassifier(), random_state=42),
                                    param_distributions=param_dist_bag_clf,
                                    n_iter=30,
                                    cv=3,
                                    scoring='f1',
                                    random_state=42,
                                    n_jobs=-1)

random_bag_clf.fit(X_small_bag_clf, y_small_bag_clf)
best_params_random_bag_clf = random_bag_clf.best_params_
print("Лучшие параметры от RandomizedSearchCV:", best_params_random_bag_clf)

In [ ]:
def objective_bag_clf(trial):
    clf = BaggingClassifier(
        estimator=RandomForestClassifier(),
        n_estimators=trial.suggest_int('n_estimators', 10, 100),
        max_samples=trial.suggest_float('max_samples', 0.5, 1.0),
        max_features=trial.suggest_float('max_features', 0.5, 1.0),
        bootstrap=trial.suggest_categorical('bootstrap', [True, False]),
        random_state=42
    )
    return cross_val_score(clf, X_small_bag_clf, y_small_bag_clf, cv=3, scoring='f1').mean()

study_bag_clf = optuna.create_study(direction='maximize')
study_bag_clf.optimize(objective_bag_clf, n_trials=30)
best_params_optuna_bag_clf = study_bag_clf.best_params
print("Лучшие параметры от Optuna:", best_params_optuna_bag_clf)

In [ ]:
sample_size = len(X_train_clf) // 20
X_train_small = X_train_clf[:sample_size]
y_train_small = y_train_clf[:sample_size]

In [ ]:
grid_model_bag_clf = BaggingClassifier(estimator=RandomForestClassifier(), random_state=42, **best_params_grid_bag_clf)
grid_model_bag_clf.fit(X_train_small, y_train_small)

In [ ]:
random_model_bag_clf = BaggingClassifier(estimator=RandomForestClassifier(), random_state=42, **best_params_random_bag_clf)
random_model_bag_clf.fit(X_train_small, y_train_small)

In [ ]:
optuna_model_bag_clf = BaggingClassifier(estimator=RandomForestClassifier(), random_state=42, **best_params_optuna_bag_clf)
optuna_model_bag_clf.fit(X_train_small, y_train_small)

In [ ]:
y_pred_grid_bag_clf = grid_model_bag_clf.predict(X_test_clf)
y_pred_rand_bag_clf = random_model_bag_clf.predict(X_test_clf)
y_pred_optuna_bag_clf = optuna_model_bag_clf.predict(X_test_clf)
comparison_bag_clf = pd.DataFrame({
    "Метрика": ["F1", "Precision", "Recall", "Accuracy"],
    "GridSearch": [
        f1_score(y_test_clf, y_pred_grid_bag_clf),
        precision_score(y_test_clf, y_pred_grid_bag_clf),
        recall_score(y_test_clf, y_pred_grid_bag_clf),
        accuracy_score(y_test_clf, y_pred_grid_bag_clf)
    ],
    "RandomSearch": [
        f1_score(y_test_clf, y_pred_rand_bag_clf),
        precision_score(y_test_clf, y_pred_rand_bag_clf),
        recall_score(y_test_clf, y_pred_rand_bag_clf),
        accuracy_score(y_test_clf, y_pred_rand_bag_clf)
    ],
    "Optuna": [
        f1_score(y_test_clf, y_pred_optuna_bag_clf),
        precision_score(y_test_clf, y_pred_optuna_bag_clf),
        recall_score(y_test_clf, y_pred_optuna_bag_clf),
        accuracy_score(y_test_clf, y_pred_optuna_bag_clf)
    ]
})
print(comparison_bag_clf)

In [ ]:
final_bag_clf = grid_model_bag_clf
# Метрики финальной
y_final_pred_bag_clf = final_bag_clf.predict(X_test_clf)
clf_metrics = {
    "Метрика": ["Accuracy", "Precision", "Recall", "F1-score"],
    "Значение": [
        accuracy_score(y_test_clf, y_final_pred_bag_clf),
        precision_score(y_test_clf, y_final_pred_bag_clf),
        recall_score(y_test_clf, y_final_pred_bag_clf),
        f1_score(y_test_clf, y_final_pred_bag_clf)
    ]
}

clf_metrics_bag_df = pd.DataFrame(clf_metrics)
print("\nКлассификационный отчёт финальной модели (BaggingClassifier):")
print(clf_metrics_bag_df)

### BaggingRegressor

In [ ]:
X_small_bag_reg = X_reg.sample(frac=0.001, random_state=42)
y_small_bag_reg = y_reg[X_small_bag_reg.index]

In [ ]:
from sklearn.ensemble import BaggingRegressor

In [ ]:
param_grid_bag_reg = {
    'n_estimators': [10, 30, 50],
    'max_samples': [0.5, 0.7, 1.0],
    'max_features': [0.5, 0.7, 1.0],
    'bootstrap': [True, False]
}

grid_bag_reg = GridSearchCV(
    BaggingRegressor(estimator=RandomForestRegressor(), random_state=42),
    param_grid=param_grid_bag_reg,
    cv=3,
    scoring='neg_mean_squared_error', 
    n_jobs=-1
)

grid_bag_reg.fit(X_small_bag_reg, y_small_bag_reg)
best_params_grid_bag_reg = grid_bag_reg.best_params_
print("Лучшие параметры для BaggingRegressor от GridSearch:", best_params_grid_bag_reg)

In [ ]:
param_dist_bag_reg = {
    'n_estimators': [10, 30, 50],
    'max_samples': [0.5, 0.7, 1.0],
    'max_features': [0.5, 0.7, 1.0],
    'bootstrap': [True, False]
}

random_bag_reg = RandomizedSearchCV(
    BaggingRegressor(estimator=RandomForestRegressor(), random_state=42),
    param_distributions=param_dist_bag_reg,
    n_iter=30,
    cv=3,
    scoring='neg_mean_squared_error',
    random_state=42,
    n_jobs=-1
)

random_bag_reg.fit(X_small_bag_reg, y_small_bag_reg)
best_params_random_bag_reg = random_bag_reg.best_params_
print("Лучшие параметры для BaggingRegressor от RandomizedSearch:", best_params_random_bag_reg)

In [ ]:
def objective_bag_reg(trial):
    bag_reg = BaggingRegressor(
        estimator=RandomForestRegressor(),
        n_estimators=trial.suggest_int('n_estimators', 10, 50),
        max_samples=trial.suggest_uniform('max_samples', 0.5, 1.0),
        max_features=trial.suggest_uniform('max_features', 0.5, 1.0),
        bootstrap=trial.suggest_categorical('bootstrap', [True, False]),
        random_state=42
    )
    score = cross_val_score(bag_reg, X_small_bag_reg, y_small_bag_reg, cv=3, scoring='neg_mean_squared_error').mean()
    return score

study_bag_reg = optuna.create_study(direction='maximize')
study_bag_reg.optimize(objective_bag_reg, n_trials=30)
best_params_optuna_bag_reg = study_bag_reg.best_params
print("Лучшие параметры для BaggingRegressor от Optuna:", best_params_optuna_bag_reg)


In [ ]:
X_train_small_reg = X_train_reg.sample(frac=0.1, random_state=42)
y_train_small_reg = y_train_reg.loc[X_train_small_reg.index]

In [ ]:
# Обучение модели с лучшими параметрами от GridSearchCV
best_bag_reg_grid = BaggingRegressor(estimator=RandomForestRegressor(), random_state=42, **best_params_grid_bag_reg)
best_bag_reg_grid.fit(X_train_small_reg, y_train_small_reg)

In [ ]:
# Обучение модели с лучшими параметрами от RandomizedSearchCV
best_bag_reg_random = BaggingRegressor(estimator=RandomForestRegressor(), random_state=42, **best_params_random_bag_reg)
best_bag_reg_random.fit(X_train_small_reg, y_train_small_reg)

In [ ]:
# Обучение модели с лучшими параметрами от Optuna
best_bag_reg_optuna = BaggingRegressor(estimator=RandomForestRegressor(), random_state=42, **best_params_optuna_bag_reg)
best_bag_reg_optuna.fit(X_train_small_reg, y_train_small_reg)

In [ ]:
y_pred_grid_bag_reg = best_bag_reg_grid.predict(X_test_reg)
y_pred_random_bag_reg = best_bag_reg_random.predict(X_test_reg)
y_pred_optuna_bag_reg = best_bag_reg_optuna.predict(X_test_reg)
comparison_bag_reg = pd.DataFrame({
    "Метрика": ["MSE", "MAE", "R2", "MAPE"],
    "GridSearch": [
        mean_squared_error(y_test_reg, y_pred_grid_bag_reg),
        mean_absolute_error(y_test_reg, y_pred_grid_bag_reg),
        r2_score(y_test_reg, y_pred_grid_bag_reg),
        mean_absolute_percentage_error(y_test_reg, y_pred_grid_bag_reg)
    ],
    "RandomSearch": [
        mean_squared_error(y_test_reg, y_pred_random_bag_reg),
        mean_absolute_error(y_test_reg, y_pred_random_bag_reg),
        r2_score(y_test_reg, y_pred_random_bag_reg),
        mean_absolute_percentage_error(y_test_reg, y_pred_random_bag_reg)
    ],
    "Optuna": [
        mean_squared_error(y_test_reg, y_pred_optuna_bag_reg),
        mean_absolute_error(y_test_reg, y_pred_optuna_bag_reg),
        r2_score(y_test_reg, y_pred_optuna_bag_reg),
        mean_absolute_percentage_error(y_test_reg, y_pred_optuna_bag_reg)
    ]
})

print("\nСравнение методов подбора для BaggingRegressor:")
print(comparison_bag_reg)

In [ ]:
comparison_final_bag_reg_random = pd.DataFrame({
    "Метрика": ["MSE", "MAE", "R2", "MAPE"],
    "RandomSearch": [
        mean_squared_error(y_test_reg, y_pred_random_bag_reg),
        mean_absolute_error(y_test_reg, y_pred_random_bag_reg),
        r2_score(y_test_reg, y_pred_random_bag_reg),
        mean_absolute_percentage_error(y_test_reg, y_pred_random_bag_reg)
    ]
})

print("\nМетрики для модели BaggingRegressor с RandomizedSearchCV:")
print(comparison_final_bag_reg_random)

## Ансамбли из Scikit-learn: GradientBoostingRegressor GradientBoostingClassifier

Решите задачу регрессии и классификации на ваших данных с использованием ансамблей из Scikit-learn: 
- GradientBoostingRegressor 
- GradientBoostingClassifier

### GradientBoostingClassifier

In [ ]:
X_small_gb_clf, _, y_small_gb_clf, _ = train_test_split(X_train_clf, y_train_clf, train_size=0.002, stratify=y_train_clf, random_state=42)

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

In [ ]:
param_grid_gb_clf = {
    'n_estimators': [50, 100],
    'learning_rate': [0.05, 0.1],
    'max_depth': [3, 5],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 2]
}

grid_gb_clf = GridSearchCV(GradientBoostingClassifier(random_state=42), param_grid_gb_clf,
                           cv=3, scoring='f1', n_jobs=-1)
grid_gb_clf.fit(X_small_gb_clf, y_small_gb_clf)
best_params_grid_gb_clf = grid_gb_clf.best_params_

In [ ]:
param_dist_gb_clf = {
    'n_estimators': randint(50, 150),
    'learning_rate': uniform(0.01, 0.2),
    'max_depth': randint(3, 10),
    'min_samples_split': randint(2, 10),
    'min_samples_leaf': randint(1, 5)
}

rand_gb_clf = RandomizedSearchCV(GradientBoostingClassifier(random_state=42), param_distributions=param_dist_gb_clf,
                                 n_iter=30, cv=3, scoring='f1', n_jobs=-1, random_state=42)
rand_gb_clf.fit(X_small_gb_clf, y_small_gb_clf)
best_params_rand_gb_clf = rand_gb_clf.best_params_

In [ ]:
def objective_gb_clf(trial):
    clf = GradientBoostingClassifier(
        n_estimators=trial.suggest_int('n_estimators', 50, 150),
        learning_rate=trial.suggest_float('learning_rate', 0.01, 0.2),
        max_depth=trial.suggest_int('max_depth', 3, 10),
        min_samples_split=trial.suggest_int('min_samples_split', 2, 10),
        min_samples_leaf=trial.suggest_int('min_samples_leaf', 1, 5),
        random_state=42
    )
    score = cross_val_score(clf, X_small_gb_clf, y_small_gb_clf, cv=3, scoring='f1').mean()
    return score

study_gb_clf = optuna.create_study(direction='maximize')
study_gb_clf.optimize(objective_gb_clf, n_trials=30)
best_params_optuna_gb_clf = study_gb_clf.best_params

In [ ]:
grid_gb_clf_final = GradientBoostingClassifier(**best_params_grid_gb_clf, random_state=42)
grid_gb_clf_final.fit(X_train_clf, y_train_clf)

rand_gb_clf_final = GradientBoostingClassifier(**best_params_rand_gb_clf, random_state=42)
rand_gb_clf_final.fit(X_train_clf, y_train_clf)

optuna_gb_clf_final = GradientBoostingClassifier(**best_params_optuna_gb_clf, random_state=42)
optuna_gb_clf_final.fit(X_train_clf, y_train_clf)

In [ ]:
def get_metrics(model, X_test, y_test):
    y_pred = model.predict(X_test)
    return {
        'Accuracy': accuracy_score(y_test, y_pred),
        'Precision': precision_score(y_test, y_pred),
        'Recall': recall_score(y_test, y_pred),
        'F1': f1_score(y_test, y_pred)
    }

metrics_gb_clf = pd.DataFrame({
    'Метрика': ['Accuracy', 'Precision', 'Recall', 'F1'],
    'GridSearch': list(get_metrics(grid_gb_clf_final, X_test_clf, y_test_clf).values()),
    'RandomSearch': list(get_metrics(rand_gb_clf_final, X_test_clf, y_test_clf).values()),
    'Optuna': list(get_metrics(optuna_gb_clf_final, X_test_clf, y_test_clf).values())
})
print("\nСравнение метрик для GradientBoostingClassifier:")
print(metrics_gb_clf)

In [ ]:
metrics_gb_clff = pd.DataFrame({
    'Метрика': ['Accuracy', 'Precision', 'Recall', 'F1'],
    'GridSearch': list(get_metrics(grid_gb_clf_final, X_test_clf, y_test_clf).values()),
})
print("\nЛучшая метрика GridSearch:")
print(metrics_gb_clff)

### GradientBoostingRegressor

In [ ]:
X_small_gb_reg = X_train_reg.sample(frac=0.01, random_state=42)
y_small_gb_reg = y_train_reg.loc[X_small_gb_reg.index]

In [ ]:
param_grid_gb_reg = {
    'n_estimators': [50, 100, 150],
    'learning_rate': [0.01, 0.05, 0.1],
    'max_depth': [3, 5, 7],
    'subsample': [0.7, 0.9, 1.0]
}
grid_gb_reg = GridSearchCV(GradientBoostingRegressor(random_state=42),
                           param_grid=param_grid_gb_reg,
                           cv=3,
                           scoring='neg_mean_squared_error',
                           n_jobs=-1)
grid_gb_reg.fit(X_small_gb_reg, y_small_gb_reg)
best_params_grid_gb_reg = grid_gb_reg.best_params_

In [ ]:
param_dist_gb_reg = {
    'n_estimators': randint(50, 200),
    'learning_rate': uniform(0.01, 0.1),
    'max_depth': randint(3, 10),
    'subsample': uniform(0.7, 1.0)
}
rand_gb_reg = RandomizedSearchCV(GradientBoostingRegressor(random_state=42),
                                 param_distributions=param_dist_gb_reg,
                                 n_iter=30,
                                 cv=3,
                                 scoring='neg_mean_squared_error',
                                 random_state=42,
                                 n_jobs=-1)
rand_gb_reg.fit(X_small_gb_reg, y_small_gb_reg)
best_params_rand_gb_reg = rand_gb_reg.best_params_

In [ ]:
def objective_gb_reg(trial):
    reg = GradientBoostingRegressor(
        n_estimators=trial.suggest_int("n_estimators", 50, 200),
        learning_rate=trial.suggest_float("learning_rate", 0.01, 0.1),
        max_depth=trial.suggest_int("max_depth", 3, 10),
        subsample=trial.suggest_float("subsample", 0.7, 1.0),
        random_state=42
    )
    return cross_val_score(reg, X_small_gb_reg, y_small_gb_reg, cv=3, scoring='neg_mean_squared_error').mean()

study_gb_reg = optuna.create_study(direction='maximize')
study_gb_reg.optimize(objective_gb_reg, n_trials=30)
best_params_optuna_gb_reg = study_gb_reg.best_params

In [ ]:
grid_gb_reg_final = GradientBoostingRegressor(**best_params_grid_gb_reg, random_state=42)
rand_gb_reg_final = GradientBoostingRegressor(**best_params_rand_gb_reg, random_state=42)
optuna_gb_reg_final = GradientBoostingRegressor(**best_params_optuna_gb_reg, random_state=42)

grid_gb_reg_final.fit(X_train_reg, y_train_reg)
rand_gb_reg_final.fit(X_train_reg, y_train_reg)
optuna_gb_reg_final.fit(X_train_reg, y_train_reg)

In [ ]:
y_pred_grid_gb_reg = grid_gb_reg_final.predict(X_test_reg)
y_pred_rand_gb_reg = rand_gb_reg_final.predict(X_test_reg)
y_pred_optuna_gb_reg = optuna_gb_reg_final.predict(X_test_reg)

metrics_gb_reg = pd.DataFrame({
    "Метрика": ["MSE", "MAE", "R2", "MAPE"],
    "GridSearch": [
        mean_squared_error(y_test_reg, y_pred_grid_gb_reg),
        mean_absolute_error(y_test_reg, y_pred_grid_gb_reg),
        r2_score(y_test_reg, y_pred_grid_gb_reg),
        mean_absolute_percentage_error(y_test_reg, y_pred_grid_gb_reg)
    ],
    "RandomSearch": [
        mean_squared_error(y_test_reg, y_pred_rand_gb_reg),
        mean_absolute_error(y_test_reg, y_pred_rand_gb_reg),
        r2_score(y_test_reg, y_pred_rand_gb_reg),
        mean_absolute_percentage_error(y_test_reg, y_pred_rand_gb_reg)
    ],
    "Optuna": [
        mean_squared_error(y_test_reg, y_pred_optuna_gb_reg),
        mean_absolute_error(y_test_reg, y_pred_optuna_gb_reg),
        r2_score(y_test_reg, y_pred_optuna_gb_reg),
        mean_absolute_percentage_error(y_test_reg, y_pred_optuna_gb_reg)
    ]
})

print("\nСравнение моделей GradientBoostingRegressor:")
print(metrics_gb_reg)

In [ ]:
metrics_gb_regf = pd.DataFrame({
    "Optuna": [
        mean_squared_error(y_test_reg, y_pred_optuna_gb_reg),
        mean_absolute_error(y_test_reg, y_pred_optuna_gb_reg),
        r2_score(y_test_reg, y_pred_optuna_gb_reg),
        mean_absolute_percentage_error(y_test_reg, y_pred_optuna_gb_reg)
    ]
})

print("\nЛучшая метрика для GradientBoostingRegressor:")
print(metrics_gb_regf)

## Ансамбли из Scikit-learn: StackingRegressor StackingClassifier 

Решите задачу регрессии и классификации на ваших данных с использованием ансамблей из Scikit-learn: 
- StackingRegressor 
- StackingClassifier 

### StackingClassifier 

In [ ]:
X_small_stack_clf = X_train_clf.sample(frac=0.01, random_state=42)
y_small_stack_clf = y_train_clf.loc[X_small_stack_clf.index]

In [ ]:
estimators = [
    ('dt', DecisionTreeClassifier(random_state=42)),
    ('rf', RandomForestClassifier(random_state=42)),
    ('lr', LogisticRegression(random_state=42, max_iter=1000))
]

param_grid_stack_clf = {
    'cv': [3, 5],
    'stack_method': ['auto', 'predict_proba'],
    'passthrough': [True, False]
}

grid_stack_clf = GridSearchCV(
    StackingClassifier(estimators=estimators, final_estimator=LogisticRegression(), n_jobs=-1),
    param_grid_stack_clf,
    scoring='f1',
    cv=3,
    n_jobs=-1
)

grid_stack_clf.fit(X_small_stack_clf, y_small_stack_clf)
best_params_grid_stack_clf = grid_stack_clf.best_params_
print("GridSearch лучшие параметры:", best_params_grid_stack_clf)

In [ ]:
param_dist_stack_clf = {
    'cv': [3, 4, 5],
    'stack_method': ['auto', 'predict_proba'],
    'passthrough': [True, False]
}

random_stack_clf = RandomizedSearchCV(
    StackingClassifier(estimators=estimators, final_estimator=LogisticRegression(), n_jobs=-1),
    param_distributions=param_dist_stack_clf,
    n_iter=10,
    cv=3,
    scoring='f1',
    random_state=42,
    n_jobs=-1
)

random_stack_clf.fit(X_small_stack_clf, y_small_stack_clf)
best_params_random_stack_clf = random_stack_clf.best_params_
print("RandomizedSearch лучшие параметры:", best_params_random_stack_clf)

In [ ]:
def objective_stack_clf(trial):
    clf = StackingClassifier(
        estimators=estimators,
        final_estimator=LogisticRegression(),
        cv=trial.suggest_categorical('cv', [3, 4, 5]),
        stack_method=trial.suggest_categorical('stack_method', ['auto', 'predict_proba']),
        passthrough=trial.suggest_categorical('passthrough', [True, False]),
        n_jobs=-1
    )
    return cross_val_score(clf, X_small_stack_clf, y_small_stack_clf, scoring='f1', cv=3).mean()

study_stack_clf = optuna.create_study(direction='maximize')
study_stack_clf.optimize(objective_stack_clf, n_trials=10)
best_params_optuna_stack_clf = study_stack_clf.best_params
print("Optuna лучшие параметры:", best_params_optuna_stack_clf)

In [ ]:
final_stack_grid_clf = StackingClassifier(**best_params_grid_stack_clf, estimators=estimators, final_estimator=LogisticRegression(), n_jobs=-1)
final_stack_grid_clf.fit(X_train_clf, y_train_clf)

final_stack_random_clf = StackingClassifier(**best_params_random_stack_clf, estimators=estimators, final_estimator=LogisticRegression(), n_jobs=-1)
final_stack_random_clf.fit(X_train_clf, y_train_clf)

final_stack_optuna_clf = StackingClassifier(**best_params_optuna_stack_clf, estimators=estimators, final_estimator=LogisticRegression(), n_jobs=-1)
final_stack_optuna_clf.fit(X_train_clf, y_train_clf)

In [ ]:
y_pred_grid = final_stack_grid_clf.predict(X_test_clf)
y_pred_random = final_stack_random_clf.predict(X_test_clf)
y_pred_optuna = final_stack_optuna_clf.predict(X_test_clf)

comparison_stack_clf = pd.DataFrame({
    "Метрика": ["Accuracy", "Precision", "Recall", "F1"],
    "GridSearch": [
        accuracy_score(y_test_clf, y_pred_grid),
        precision_score(y_test_clf, y_pred_grid),
        recall_score(y_test_clf, y_pred_grid),
        f1_score(y_test_clf, y_pred_grid)
    ],
    "RandomSearch": [
        accuracy_score(y_test_clf, y_pred_random),
        precision_score(y_test_clf, y_pred_random),
        recall_score(y_test_clf, y_pred_random),
        f1_score(y_test_clf, y_pred_random)
    ],
    "Optuna": [
        accuracy_score(y_test_clf, y_pred_optuna),
        precision_score(y_test_clf, y_pred_optuna),
        recall_score(y_test_clf, y_pred_optuna),
        f1_score(y_test_clf, y_pred_optuna)
    ]
})

print("Сравнение моделей StackingClassifier:")
print(comparison_stack_clf)

In [ ]:
comparison_stack_clff = pd.DataFrame({
    "RandomSearch": [
        accuracy_score(y_test_clf, y_pred_random),
        precision_score(y_test_clf, y_pred_random),
        recall_score(y_test_clf, y_pred_random),
        f1_score(y_test_clf, y_pred_random)
    ]
})

print("Лучшая метрика для StackingClassifier:")
print(comparison_stack_clff)

### StackingRegressor

In [ ]:
X_small_stack_reg = X_train_reg.sample(frac=0.01, random_state=42)
y_small_stack_reg = y_train_reg.loc[X_small_stack_reg.index]

In [ ]:
regressors = [
    ('dt', DecisionTreeClassifier(random_state=42)),
    ('rf', RandomForestClassifier(random_state=42)),
    ('lr', LogisticRegression(random_state=42, max_iter=1000))
]
]

param_grid_stack_reg = {
    'cv': [3, 5],
    'passthrough': [True, False]
}

grid_stack_reg = GridSearchCV(
    StackingRegressor(estimators=regressors, final_estimator=Ridge(), n_jobs=-1),
    param_grid=param_grid_stack_reg,
    cv=3,
    scoring='r2',
    n_jobs=-1
)

grid_stack_reg.fit(X_small_stack_reg, y_small_stack_reg)
best_params_grid_stack_reg = grid_stack_reg.best_params_
print("GridSearch лучшие параметры:", best_params_grid_stack_reg)

In [ ]:
param_dist_stack_reg = {
    'cv': [3, 4, 5],
    'passthrough': [True, False]
}

random_stack_reg = RandomizedSearchCV(
    StackingRegressor(estimators=regressors, final_estimator=Ridge(), n_jobs=-1),
    param_distributions=param_dist_stack_reg,
    n_iter=10,
    cv=3,
    scoring='r2',
    random_state=42,
    n_jobs=-1
)

random_stack_reg.fit(X_small_stack_reg, y_small_stack_reg)
best_params_random_stack_reg = random_stack_reg.best_params_
print("RandomizedSearch лучшие параметры:", best_params_random_stack_reg)

In [ ]:
def objective_stack_reg(trial):
    reg = StackingRegressor(
        estimators=regressors,
        final_estimator=Ridge(),
        cv=trial.suggest_categorical('cv', [3, 4, 5]),
        passthrough=trial.suggest_categorical('passthrough', [True, False]),
        n_jobs=-1
    )
    return cross_val_score(reg, X_small_stack_reg, y_small_stack_reg, scoring='r2', cv=3).mean()

study_stack_reg = optuna.create_study(direction='maximize')
study_stack_reg.optimize(objective_stack_reg, n_trials=10)
best_params_optuna_stack_reg = study_stack_reg.best_params
print("Optuna лучшие параметры:", best_params_optuna_stack_reg)

In [ ]:
final_stack_grid_reg = StackingRegressor(**best_params_grid_stack_reg, estimators=regressors, final_estimator=Ridge(), n_jobs=-1)
final_stack_grid_reg.fit(X_train_reg, y_train_reg)

final_stack_random_reg = StackingRegressor(**best_params_random_stack_reg, estimators=regressors, final_estimator=Ridge(), n_jobs=-1)
final_stack_random_reg.fit(X_train_reg, y_train_reg)

final_stack_optuna_reg = StackingRegressor(**best_params_optuna_stack_reg, estimators=regressors,final_estimator=Ridge(), n_jobs=-1)
final_stack_optuna_reg.fit(X_train_reg, y_train_reg)

In [ ]:
y_pred_grid_reg = final_stack_grid_reg.predict(X_test_reg)
y_pred_random_reg = final_stack_random_reg.predict(X_test_reg)
y_pred_optuna_reg = final_stack_optuna_reg.predict(X_test_reg)

comparison_stack_reg = pd.DataFrame({
    "Метрика": ["MSE", "MAE", "R2", "MAPE"],
    "GridSearch": [
        mean_squared_error(y_test_reg, y_pred_grid_reg),
        mean_absolute_error(y_test_reg, y_pred_grid_reg),
        r2_score(y_test_reg, y_pred_grid_reg),
        mean_absolute_percentage_error(y_test_reg, y_pred_grid_reg)
    ],
    "RandomSearch": [
        mean_squared_error(y_test_reg, y_pred_random_reg),
        mean_absolute_error(y_test_reg, y_pred_random_reg),
        r2_score(y_test_reg, y_pred_random_reg),
        mean_absolute_percentage_error(y_test_reg, y_pred_random_reg)
    ],
    "Optuna": [
        mean_squared_error(y_test_reg, y_pred_optuna_reg),
        mean_absolute_error(y_test_reg, y_pred_optuna_reg),
        r2_score(y_test_reg, y_pred_optuna_reg),
        mean_absolute_percentage_error(y_test_reg, y_pred_optuna_reg)
    ]
})

print("Сравнение моделей StackingRegressor:")
print(comparison_stack_reg)

In [ ]:
comparison_stack_regf = pd.DataFrame({
    "Метрика": ["MSE", "MAE", "R2", "MAPE"],
    "GridSearch": [
        mean_squared_error(y_test_reg, y_pred_grid_reg),
        mean_absolute_error(y_test_reg, y_pred_grid_reg),
        r2_score(y_test_reg, y_pred_grid_reg),
        mean_absolute_percentage_error(y_test_reg, y_pred_grid_reg)
    ]
})

print("Лучшая метрика для StackingRegressor:")
print(comparison_stack_regf)

## Градиентный бустинг над деревьями решений: CatBoost

Решите задачу регрессии и классификации на ваших данных с использованием библиотек градиентного бустинга над деревьями решений: 
- CatBoost

### CatBoost.Классификация

In [ ]:
X_small_cat_clf, _, y_small_cat_clf, _ = train_test_split(X_train_clf, y_train_clf, train_size=0.01, random_state=42, stratify=y_train_clf)

In [ ]:
param_grid_cat_clf = {
    'iterations': [100, 200],
    'depth': [4, 6, 8],
    'learning_rate': [0.01, 0.05, 0.1],
    'l2_leaf_reg': [1, 3, 5]
}

grid_cat_clf = GridSearchCV(
    CatBoostClassifier(verbose=0, random_state=42),
    param_grid_cat_clf,
    cv=3,
    scoring='f1',
    n_jobs=-1
)
grid_cat_clf.fit(X_small_cat_clf, y_small_cat_clf)
best_params_grid_cat_clf = grid_cat_clf.best_params_

In [ ]:
param_dist_cat_clf = {
    'iterations': randint(100, 300),
    'depth': randint(3, 10),
    'learning_rate': uniform(0.01, 0.2),
    'l2_leaf_reg': randint(1, 10)
}

random_cat_clf = RandomizedSearchCV(
    CatBoostClassifier(verbose=0, random_state=42),
    param_distributions=param_dist_cat_clf,
    n_iter=30,
    cv=3,
    scoring='f1',
    random_state=42,
    n_jobs=-1
)
random_cat_clf.fit(X_small_cat_clf, y_small_cat_clf)
best_params_random_cat_clf = random_cat_clf.best_params_

In [ ]:
def objective_cat_clf(trial):
    model = CatBoostClassifier(
        iterations=trial.suggest_int("iterations", 100, 300),
        depth=trial.suggest_int("depth", 3, 10),
        learning_rate=trial.suggest_float("learning_rate", 0.01, 0.2),
        l2_leaf_reg=trial.suggest_int("l2_leaf_reg", 1, 10),
        verbose=0,
        random_state=42
    )
    score = cross_val_score(model, X_small_cat_clf, y_small_cat_clf, cv=3, scoring='f1').mean()
    return score

study_cat_clf = optuna.create_study(direction="maximize")
study_cat_clf.optimize(objective_cat_clf, n_trials=30)
best_params_optuna_cat_clf = study_cat_clf.best_params

In [ ]:
final_cat_grid_clf = CatBoostClassifier(**best_params_grid_cat_clf, verbose=0, random_state=42)
final_cat_grid_clf.fit(X_train_clf, y_train_clf)

final_cat_random_clf = CatBoostClassifier(**best_params_random_cat_clf, verbose=0, random_state=42)
final_cat_random_clf.fit(X_train_clf, y_train_clf)

final_cat_optuna_clf = CatBoostClassifier(**best_params_optuna_cat_clf, verbose=0, random_state=42)
final_cat_optuna_clf.fit(X_train_clf, y_train_clf)

In [ ]:
pred_grid = final_cat_grid_clf.predict(X_test_clf)
pred_rand = final_cat_random_clf.predict(X_test_clf)
pred_optuna = final_cat_optuna_clf.predict(X_test_clf)

comparison_cat_clf = pd.DataFrame({
    "Метрика": ["Accuracy", "Precision", "Recall", "F1"],
    "GridSearch": [
        accuracy_score(y_test_clf, pred_grid),
        precision_score(y_test_clf, pred_grid),
        recall_score(y_test_clf, pred_grid),
        f1_score(y_test_clf, pred_grid)
    ],
    "RandomSearch": [
        accuracy_score(y_test_clf, pred_rand),
        precision_score(y_test_clf, pred_rand),
        recall_score(y_test_clf, pred_rand),
        f1_score(y_test_clf, pred_rand)
    ],
    "Optuna": [
        accuracy_score(y_test_clf, pred_optuna),
        precision_score(y_test_clf, pred_optuna),
        recall_score(y_test_clf, pred_optuna),
        f1_score(y_test_clf, pred_optuna)
    ]
})

print("Сравнение CatBoostClassifier:")
print(comparison_cat_clf)

In [ ]:
comparison_cat_clf = pd.DataFrame({
    "Метрика": ["Accuracy", "Precision", "Recall", "F1"],
    "RandomSearch": [
        accuracy_score(y_test_clf, pred_rand),
        precision_score(y_test_clf, pred_rand),
        recall_score(y_test_clf, pred_rand),
        f1_score(y_test_clf, pred_rand)
    ]
})

print("Лучшая метрика для CatBoostClassifier:")
print(comparison_cat_clf)

### CatBoost. Регрессия

In [ ]:
X_small_cat_reg, _, y_small_cat_reg, _ = train_test_split(X_train_reg, y_train_reg,
                                                          train_size=0.01,
                                                          random_state=42)

In [ ]:
param_grid_cat_reg = {
    'iterations': [100, 200],
    'learning_rate': [0.01, 0.05, 0.1],
    'depth': [4, 6, 8],
    'l2_leaf_reg': [1, 3, 5]
}
grid_cat_reg = GridSearchCV(
    CatBoostRegressor(verbose=0, random_state=42),
    param_grid=param_grid_cat_reg,
    cv=3,
    scoring='neg_mean_squared_error',
    n_jobs=-1
)
grid_cat_reg.fit(X_small_cat_reg, y_small_cat_reg)
best_params_grid_cat_reg = grid_cat_reg.best_params_
print("GridSearch лучшие параметры (CatBoostRegressor):", best_params_grid_cat_reg)

In [ ]:
param_dist_cat_reg = {
    'iterations': randint(100, 300),
    'learning_rate': uniform(0.01, 0.1),
    'depth': randint(3, 10),
    'l2_leaf_reg': randint(1, 10)
}
random_cat_reg = RandomizedSearchCV(
    CatBoostRegressor(verbose=0, random_state=42),
    param_distributions=param_dist_cat_reg,
    n_iter=30,
    cv=3,
    scoring='neg_mean_squared_error',
    random_state=42,
    n_jobs=-1
)
random_cat_reg.fit(X_small_cat_reg, y_small_cat_reg)
best_params_random_cat_reg = random_cat_reg.best_params_
print("RandomizedSearch лучшие параметры (CatBoostRegressor):", best_params_random_cat_reg)

In [ ]:
def objective_cat_reg(trial):
    model = CatBoostRegressor(
        iterations=trial.suggest_int("iterations", 100, 300),
        learning_rate=trial.suggest_float("learning_rate", 0.01, 0.1),
        depth=trial.suggest_int("depth", 3, 10),
        l2_leaf_reg=trial.suggest_int("l2_leaf_reg", 1, 10),
        verbose=0,
        random_state=42
    )
    score = cross_val_score(model, X_small_cat_reg, y_small_cat_reg, cv=3, scoring="neg_mean_squared_error").mean()
    return score

study_cat_reg = optuna.create_study(direction="maximize")
study_cat_reg.optimize(objective_cat_reg, n_trials=30)
best_params_optuna_cat_reg = study_cat_reg.best_params
print("Optuna лучшие параметры (CatBoostRegressor):", best_params_optuna_cat_reg)

In [ ]:
final_cat_grid_reg = CatBoostRegressor(**best_params_grid_cat_reg, verbose=0, random_state=42)
final_cat_grid_reg.fit(X_train_reg, y_train_reg)

final_cat_random_reg = CatBoostRegressor(**best_params_random_cat_reg, verbose=0, random_state=42)
final_cat_random_reg.fit(X_train_reg, y_train_reg)

final_cat_optuna_reg = CatBoostRegressor(**best_params_optuna_cat_reg, verbose=0, random_state=42)
final_cat_optuna_reg.fit(X_train_reg, y_train_reg)


In [ ]:
y_pred_grid_cat_reg = final_cat_grid_reg.predict(X_test_reg)
y_pred_random_cat_reg = final_cat_random_reg.predict(X_test_reg)
y_pred_optuna_cat_reg = final_cat_optuna_reg.predict(X_test_reg)

metrics_cat_reg = pd.DataFrame({
    "Метрика": ["MSE", "MAE", "R2", "MAPE"],
    "GridSearch": [
        mean_squared_error(y_test_reg, y_pred_grid_cat_reg),
        mean_absolute_error(y_test_reg, y_pred_grid_cat_reg),
        r2_score(y_test_reg, y_pred_grid_cat_reg),
        mean_absolute_percentage_error(y_test_reg, y_pred_grid_cat_reg)
    ],
    "RandomSearch": [
        mean_squared_error(y_test_reg, y_pred_random_cat_reg),
        mean_absolute_error(y_test_reg, y_pred_random_cat_reg),
        r2_score(y_test_reg, y_pred_random_cat_reg),
        mean_absolute_percentage_error(y_test_reg, y_pred_random_cat_reg)
    ],
    "Optuna": [
        mean_squared_error(y_test_reg, y_pred_optuna_cat_reg),
        mean_absolute_error(y_test_reg, y_pred_optuna_cat_reg),
        r2_score(y_test_reg, y_pred_optuna_cat_reg),
        mean_absolute_percentage_error(y_test_reg, y_pred_optuna_cat_reg)
    ]
})

# Отключаем экспоненциальное отображение чисел:
pd.options.display.float_format = '{:,.4f}'.format

print("\nСравнение моделей CatBoostRegressor:")
print(metrics_cat_reg)

In [ ]:
metrics_cat_regf = pd.DataFrame({
    "Метрика": ["MSE", "MAE", "R2", "MAPE"],
    "GridSearch": [
        mean_squared_error(y_test_reg, y_pred_grid_cat_reg),
        mean_absolute_error(y_test_reg, y_pred_grid_cat_reg),
        r2_score(y_test_reg, y_pred_grid_cat_reg),
        mean_absolute_percentage_error(y_test_reg, y_pred_grid_cat_reg)
    ]
})

# Отключаем экспоненциальное отображение чисел:
pd.options.display.float_format = '{:,.4f}'.format

print("\ Лучшая метрика для CatBoostRegressor:")
print(metrics_cat_regf)

## Градиентный бустинг над деревьями решений: XGBoost

Решите задачу регрессии и классификации на ваших данных с использованием библиотек градиентного бустинга над деревьями решений: 
- XGBoost 

### XGBoost. Классификация

In [ ]:
X_small_xgb_clf, _, y_small_xgb_clf, _ = train_test_split(X_train_clf, y_train_clf, train_size=0.01, random_state=42, stratify=y_train_clf)

In [ ]:
param_grid_xgb_clf = {
    'n_estimators': [50, 100],
    'max_depth': [3, 5, 7],
    'learning_rate': [0.01, 0.1],
    'subsample': [0.7, 1.0],
    'colsample_bytree': [0.7, 1.0]
}

grid_xgb_clf = GridSearchCV(XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42),
                            param_grid=param_grid_xgb_clf,
                            cv=3,
                            scoring='f1',
                            n_jobs=-1)
grid_xgb_clf.fit(X_small_xgb_clf, y_small_xgb_clf)
best_params_grid_xgb_clf = grid_xgb_clf.best_params_

In [ ]:
param_dist_xgb_clf = {
    'n_estimators': randint(50, 150),
    'max_depth': randint(3, 10),
    'learning_rate': uniform(0.01, 0.3),
    'subsample': uniform(0.7, 0.3),
    'colsample_bytree': uniform(0.7, 0.3)
}

random_xgb_clf = RandomizedSearchCV(XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42),
                                    param_distributions=param_dist_xgb_clf,
                                    n_iter=30,
                                    cv=3,
                                    scoring='f1',
                                    random_state=42,
                                    n_jobs=-1)
random_xgb_clf.fit(X_small_xgb_clf, y_small_xgb_clf)
best_params_random_xgb_clf = random_xgb_clf.best_params_

In [ ]:
def objective_xgb_clf(trial):
    clf = XGBClassifier(
        n_estimators=trial.suggest_int('n_estimators', 50, 150),
        max_depth=trial.suggest_int('max_depth', 3, 10),
        learning_rate=trial.suggest_float('learning_rate', 0.01, 0.3),
        subsample=trial.suggest_float('subsample', 0.7, 1.0),
        colsample_bytree=trial.suggest_float('colsample_bytree', 0.7, 1.0),
        use_label_encoder=False,
        eval_metric='logloss',
        random_state=42
    )
    return cross_val_score(clf, X_small_xgb_clf, y_small_xgb_clf, cv=3, scoring='f1').mean()

study_xgb_clf = optuna.create_study(direction='maximize')
study_xgb_clf.optimize(objective_xgb_clf, n_trials=30)
best_params_optuna_xgb_clf = study_xgb_clf.best_params

In [ ]:
final_xgb_clf_grid = XGBClassifier(**best_params_grid_xgb_clf, use_label_encoder=False, eval_metric='logloss', random_state=42)
final_xgb_clf_random = XGBClassifier(**best_params_random_xgb_clf, use_label_encoder=False, eval_metric='logloss', random_state=42)
final_xgb_clf_optuna = XGBClassifier(**best_params_optuna_xgb_clf, use_label_encoder=False, eval_metric='logloss', random_state=42)

final_xgb_clf_grid.fit(X_train_clf, y_train_clf)
final_xgb_clf_random.fit(X_train_clf, y_train_clf)
final_xgb_clf_optuna.fit(X_train_clf, y_train_clf)

In [ ]:
y_pred_grid = final_xgb_clf_grid.predict(X_test_clf)
y_pred_random = final_xgb_clf_random.predict(X_test_clf)
y_pred_optuna = final_xgb_clf_optuna.predict(X_test_clf)

comparison_xgb_clf = pd.DataFrame({
    "Метрика": ["Accuracy", "Precision", "Recall", "F1"],
    "GridSearch": [
        accuracy_score(y_test_clf, y_pred_grid),
        precision_score(y_test_clf, y_pred_grid),
        recall_score(y_test_clf, y_pred_grid),
        f1_score(y_test_clf, y_pred_grid)
    ],
    "RandomSearch": [
        accuracy_score(y_test_clf, y_pred_random),
        precision_score(y_test_clf, y_pred_random),
        recall_score(y_test_clf, y_pred_random),
        f1_score(y_test_clf, y_pred_random)
    ],
    "Optuna": [
        accuracy_score(y_test_clf, y_pred_optuna),
        precision_score(y_test_clf, y_pred_optuna),
        recall_score(y_test_clf, y_pred_optuna),
        f1_score(y_test_clf, y_pred_optuna)
    ]
})

print("\nСравнение XGBoost Classifier:")
print(comparison_xgb_clf)

In [ ]:
comparison_xgb_clff = pd.DataFrame({
    "RandomSearch": [
        accuracy_score(y_test_clf, y_pred_random),
        precision_score(y_test_clf, y_pred_random),
        recall_score(y_test_clf, y_pred_random),
        f1_score(y_test_clf, y_pred_random)
    ]
})

print("\nЛучшая метрика для XGBoost Classifier:")
print(comparison_xgb_clff)

### XGBoost. Регрессия

In [ ]:
X_small_xgb_reg, _, y_small_xgb_reg, _ = train_test_split(X_train_reg, y_train_reg,
                                                          train_size=0.01,
                                                          random_state=42)

In [ ]:
from xgboost import XGBRegressor
param_grid_xgb_reg = {
    'n_estimators': [50, 100],
    'max_depth': [3, 5, 7],
    'learning_rate': [0.01, 0.1],
    'subsample': [0.7, 1.0],
    'colsample_bytree': [0.7, 1.0]
}

grid_xgb_reg = GridSearchCV(XGBRegressor(random_state=42),
                            param_grid=param_grid_xgb_reg,
                            cv=3,
                            scoring='r2',
                            n_jobs=-1)
grid_xgb_reg.fit(X_small_xgb_reg, y_small_xgb_reg)
best_params_grid_xgb_reg = grid_xgb_reg.best_params_

In [ ]:
param_dist_xgb_reg = {
    'n_estimators': randint(50, 150),
    'max_depth': randint(3, 10),
    'learning_rate': uniform(0.01, 0.3),
    'subsample': uniform(0.7, 0.3),
    'colsample_bytree': uniform(0.7, 0.3)
}

random_xgb_reg = RandomizedSearchCV(XGBRegressor(random_state=42),
                                    param_distributions=param_dist_xgb_reg,
                                    n_iter=30,
                                    cv=3,
                                    scoring='r2',
                                    random_state=42,
                                    n_jobs=-1)
random_xgb_reg.fit(X_small_xgb_reg, y_small_xgb_reg)
best_params_random_xgb_reg = random_xgb_reg.best_params_

In [ ]:
def objective_xgb_reg(trial):
    model = XGBRegressor(
        n_estimators=trial.suggest_int('n_estimators', 50, 150),
        max_depth=trial.suggest_int('max_depth', 3, 10),
        learning_rate=trial.suggest_float('learning_rate', 0.01, 0.3),
        subsample=trial.suggest_float('subsample', 0.7, 1.0),
        colsample_bytree=trial.suggest_float('colsample_bytree', 0.7, 1.0),
        random_state=42
    )
    return cross_val_score(model, X_small_xgb_reg, y_small_xgb_reg, cv=3, scoring='r2').mean()

study_xgb_reg = optuna.create_study(direction='maximize')
study_xgb_reg.optimize(objective_xgb_reg, n_trials=30)
best_params_optuna_xgb_reg = study_xgb_reg.best_params

In [ ]:
final_xgb_reg_grid = XGBRegressor(**best_params_grid_xgb_reg, random_state=42)
final_xgb_reg_random = XGBRegressor(**best_params_random_xgb_reg, random_state=42)
final_xgb_reg_optuna = XGBRegressor(**best_params_optuna_xgb_reg, random_state=42)

final_xgb_reg_grid.fit(X_train_reg, y_train_reg)
final_xgb_reg_random.fit(X_train_reg, y_train_reg)
final_xgb_reg_optuna.fit(X_train_reg, y_train_reg)

In [ ]:
y_pred_grid_xgb = final_xgb_reg_grid.predict(X_test_reg)
y_pred_rand_xgb = final_xgb_reg_random.predict(X_test_reg)
y_pred_optuna_xgb = final_xgb_reg_optuna.predict(X_test_reg)

comparison_xgb_reg = pd.DataFrame({
    "Метрика": ["MSE", "MAE", "R2", "MAPE"],
    "GridSearch": [
        mean_squared_error(y_test_reg, y_pred_grid_xgb),
        mean_absolute_error(y_test_reg, y_pred_grid_xgb),
        r2_score(y_test_reg, y_pred_grid_xgb),
        mean_absolute_percentage_error(y_test_reg, y_pred_grid_xgb)
    ],
    "RandomSearch": [
        mean_squared_error(y_test_reg, y_pred_rand_xgb),
        mean_absolute_error(y_test_reg, y_pred_rand_xgb),
        r2_score(y_test_reg, y_pred_rand_xgb),
        mean_absolute_percentage_error(y_test_reg, y_pred_rand_xgb)
    ],
    "Optuna": [
        mean_squared_error(y_test_reg, y_pred_optuna_xgb),
        mean_absolute_error(y_test_reg, y_pred_optuna_xgb),
        r2_score(y_test_reg, y_pred_optuna_xgb),
        mean_absolute_percentage_error(y_test_reg, y_pred_optuna_xgb)
    ]
})

# Отключим экспоненциальный формат
pd.set_option('display.float_format', '{:.4f}'.format)

print("\nСравнение XGBoost Regressor:")
print(comparison_xgb_reg)

In [ ]:
comparison_xgb_regf = pd.DataFrame({
    "Метрика": ["MSE", "MAE", "R2", "MAPE"],
    "Optuna": [
        mean_squared_error(y_test_reg, y_pred_optuna_xgb),
        mean_absolute_error(y_test_reg, y_pred_optuna_xgb),
        r2_score(y_test_reg, y_pred_optuna_xgb),
        mean_absolute_percentage_error(y_test_reg, y_pred_optuna_xgb)
    ]
})

# Отключим экспоненциальный формат
pd.set_option('display.float_format', '{:.4f}'.format)

print("\nЛучшие метрики для XGBoost Regressor:")
print(comparison_xgb_regf)

## Градиентный бустинг над деревьями решений: LightGBM

Решите задачу регрессии и классификации на ваших данных с использованием библиотек градиентного бустинга над деревьями решений: 
- LightGBM

### LightGBM. Классификация

In [ ]:
X_small_lgb_clf, _, y_small_lgb_clf, _ = train_test_split(X_train_clf, y_train_clf, train_size=0.01, stratify=y_train_clf, random_state=42)

In [ ]:
import lightgbm as lgb

params = {
    'objective': 'regression',
    'metric': 'rmse',
    'verbose': -1  # Убирает все предупреждения
}

In [ ]:
from lightgbm import LGBMClassifier
param_grid_lgb_clf = {
    'n_estimators': [50, 100],
    'learning_rate': [0.01, 0.1],
    'max_depth': [3, 5, 7],
    'num_leaves': [15, 31, 63]
}

grid_lgb_clf = GridSearchCV(LGBMClassifier(random_state=42),
                            param_grid=param_grid_lgb_clf,
                            scoring='f1',
                            cv=3,
                            n_jobs=-1)
grid_lgb_clf.fit(X_small_lgb_clf, y_small_lgb_clf)
best_params_grid_lgb_clf = grid_lgb_clf.best_params_

### Вычисление гиперпараметров, так как много ворнинг, прячу

In [ ]:
param_dist_lgb_clf = {
    'n_estimators': randint(50, 200),
    'learning_rate': uniform(0.01, 0.2),
    'max_depth': randint(3, 10),
    'num_leaves': randint(15, 64)
}

random_lgb_clf = RandomizedSearchCV(LGBMClassifier(random_state=42),
                                    param_distributions=param_dist_lgb_clf,
                                    n_iter=30,
                                    scoring='f1',
                                    cv=3,
                                    random_state=42,
                                    n_jobs=-1)
random_lgb_clf.fit(X_small_lgb_clf, y_small_lgb_clf)
best_params_random_lgb_clf = random_lgb_clf.best_params_

In [ ]:
def objective_lgb_clf(trial):
    model = LGBMClassifier(
        n_estimators=trial.suggest_int('n_estimators', 50, 200),
        learning_rate=trial.suggest_float('learning_rate', 0.01, 0.2),
        max_depth=trial.suggest_int('max_depth', 3, 10),
        num_leaves=trial.suggest_int('num_leaves', 15, 64),
        random_state=42
    )
    score = cross_val_score(model, X_small_lgb_clf, y_small_lgb_clf, scoring='f1', cv=3).mean()
    return score

study_lgb_clf = optuna.create_study(direction='maximize')
study_lgb_clf.optimize(objective_lgb_clf, n_trials=30)
best_params_optuna_lgb_clf = study_lgb_clf.best_params

In [ ]:
# Обучение по лучшим параметрам
final_lgb_clf_grid = LGBMClassifier(**best_params_grid_lgb_clf, random_state=42).fit(X_train_clf, y_train_clf)
final_lgb_clf_random = LGBMClassifier(**best_params_random_lgb_clf, random_state=42).fit(X_train_clf, y_train_clf)
final_lgb_clf_optuna = LGBMClassifier(**best_params_optuna_lgb_clf, random_state=42).fit(X_train_clf, y_train_clf)

### Продолжение

In [ ]:
y_pred_grid = final_lgb_clf_grid.predict(X_test_clf)
y_pred_random = final_lgb_clf_random.predict(X_test_clf)
y_pred_optuna = final_lgb_clf_optuna.predict(X_test_clf)

# Сравнение

metrics_lgb_clf = pd.DataFrame({
    "Метрика": ["Accuracy", "F1", "Precision", "Recall"],
    "GridSearch": [
        accuracy_score(y_test_clf, y_pred_grid),
        f1_score(y_test_clf, y_pred_grid),
        precision_score(y_test_clf, y_pred_grid, average='binary'),
        recall_score(y_test_clf, y_pred_grid, average='binary')
    ],
    "RandomSearch": [
        accuracy_score(y_test_clf, y_pred_random),
        f1_score(y_test_clf, y_pred_random),
        precision_score(y_test_clf, y_pred_random, average='binary'),
        recall_score(y_test_clf, y_pred_random, average='binary')
    ],
    "Optuna": [
        accuracy_score(y_test_clf, y_pred_optuna),
        f1_score(y_test_clf, y_pred_optuna),
        precision_score(y_test_clf, y_pred_optuna, average='binary'),
        recall_score(y_test_clf, y_pred_optuna, average='binary')
    ]
})

print("\nСравнение моделей LightGBM-классификации:")
print(metrics_lgb_clf.round(4))

In [ ]:
metrics_lgb_clff = pd.DataFrame({
    "Метрика": ["Accuracy", "F1", "Precision", "Recall"],
    "GridSearch": [
        accuracy_score(y_test_clf, y_pred_grid),
        f1_score(y_test_clf, y_pred_grid),
        precision_score(y_test_clf, y_pred_grid, average='binary'),
        recall_score(y_test_clf, y_pred_grid, average='binary')
    ]
})

print("\nЛучшие метрики для LightGBM-классификации:")
print(metrics_lgb_clff.round(4))

### LightGBM. Регрессия

In [ ]:
X_small_lgb_reg, _, y_small_lgb_reg, _ = train_test_split(X_train_reg, y_train_reg, train_size=0.01, random_state=42)

In [ ]:
from lightgbm import LGBMRegressor

#### Здесь очень длинные ворнинг. Но происходит подбор гиперпараметров для RandomizedSearchCV и GridSearchCV и Optuna. Обучение моделей

In [ ]:
param_grid_lgb_reg = {
    'n_estimators': [50, 100],
    'learning_rate': [0.01, 0.1],
    'max_depth': [3, 5, 7],
    'num_leaves': [15, 31, 63]
}

grid_lgb_reg = GridSearchCV(LGBMRegressor(random_state=42),
                            param_grid=param_grid_lgb_reg,
                            scoring='neg_mean_squared_error',
                            cv=3,
                            n_jobs=-1)
grid_lgb_reg.fit(X_small_lgb_reg, y_small_lgb_reg)
best_params_grid_lgb_reg = grid_lgb_reg.best_params_

In [ ]:
param_dist_lgb_reg = {
    'n_estimators': randint(50, 200),
    'learning_rate': uniform(0.01, 0.2),
    'max_depth': randint(3, 10),
    'num_leaves': randint(15, 64)
}

random_lgb_reg = RandomizedSearchCV(LGBMRegressor(random_state=42),
                                    param_distributions=param_dist_lgb_reg,
                                    n_iter=30,
                                    scoring='neg_mean_squared_error',
                                    cv=3,
                                    random_state=42,
                                    n_jobs=-1)
random_lgb_reg.fit(X_small_lgb_reg, y_small_lgb_reg)
best_params_random_lgb_reg = random_lgb_reg.best_params_

In [ ]:
def objective_lgb_reg(trial):
    model = LGBMRegressor(
        n_estimators=trial.suggest_int('n_estimators', 50, 200),
        learning_rate=trial.suggest_float('learning_rate', 0.01, 0.2),
        max_depth=trial.suggest_int('max_depth', 3, 10),
        num_leaves=trial.suggest_int('num_leaves', 15, 64),
        random_state=42
    )
    score = cross_val_score(model, X_small_lgb_reg, y_small_lgb_reg, scoring='neg_mean_squared_error', cv=3).mean()
    return score

study_lgb_reg = optuna.create_study(direction='maximize')
study_lgb_reg.optimize(objective_lgb_reg, n_trials=30)
best_params_optuna_lgb_reg = study_lgb_reg.best_params


In [ ]:
# Обучение по лучшим параметрам
final_lgb_reg_grid = LGBMRegressor(**best_params_grid_lgb_reg, random_state=42).fit(X_train_reg, y_train_reg)
final_lgb_reg_random = LGBMRegressor(**best_params_random_lgb_reg, random_state=42).fit(X_train_reg, y_train_reg)
final_lgb_reg_optuna = LGBMRegressor(**best_params_optuna_lgb_reg, random_state=42).fit(X_train_reg, y_train_reg)

### Продолжение

In [ ]:
# Предсказания
y_pred_grid_reg = final_lgb_reg_grid.predict(X_test_reg)
y_pred_random_reg = final_lgb_reg_random.predict(X_test_reg)
y_pred_optuna_reg = final_lgb_reg_optuna.predict(X_test_reg)

# Сравнение

metrics_lgb_reg = pd.DataFrame({
    "Метрика": ["MSE", "MAE", "R2", "MAPE"],
    "GridSearch": [
        mean_squared_error(y_test_reg, y_pred_grid_reg),
        mean_absolute_error(y_test_reg, y_pred_grid_reg),
        r2_score(y_test_reg, y_pred_grid_reg),
        mean_absolute_percentage_error(y_test_reg, y_pred_grid_reg)
    ],
    "RandomSearch": [
        mean_squared_error(y_test_reg, y_pred_random_reg),
        mean_absolute_error(y_test_reg, y_pred_random_reg),
        r2_score(y_test_reg, y_pred_random_reg),
        mean_absolute_percentage_error(y_test_reg, y_pred_random_reg)
    ],
    "Optuna": [
        mean_squared_error(y_test_reg, y_pred_optuna_reg),
        mean_absolute_error(y_test_reg, y_pred_optuna_reg),
        r2_score(y_test_reg, y_pred_optuna_reg),
        mean_absolute_percentage_error(y_test_reg, y_pred_optuna_reg)
    ]
})

print("\nСравнение моделей LightGBM-регрессии:")
print(metrics_lgb_reg.round(4))

In [ ]:
metrics_lgb_regf = pd.DataFrame({
    "Метрика": ["MSE", "MAE", "R2", "MAPE"],
    "RandomSearch": [
        mean_squared_error(y_test_reg, y_pred_random_reg),
        mean_absolute_error(y_test_reg, y_pred_random_reg),
        r2_score(y_test_reg, y_pred_random_reg),
        mean_absolute_percentage_error(y_test_reg, y_pred_random_reg)
    ]
})

print("\nЛучшие метрики для LightGBM-регрессии:")
print(metrics_lgb_regf.round(4))

# Оценка качества моделей

Выполнить оценку качества моделей ML. Сравнить полученные результаты обучения ваших моделей ML с результатами, полученными с помощью PyCaret. Обратите внимание на сгенерированные таблицы (и их содержание) как результаты работы алгоритмов ML посредством PyCaret.

In [ ]:
from pycaret.classification import *

df_clf = X_train_clf.copy()
df_clf['fraud'] = y_train_clf

clf_setup = setup(data=df_clf, target='fraud', session_id=42, verbose=False)

# Сравнение моделей
clf_models = compare_models()
clf_results = pull()
print(clf_results)

In [ ]:
from pycaret.regression import *

# Объединяем X и y в один DataFrame
df_reg = X_train_reg.copy()
df_reg['target'] = y_train_reg  # замените 'target' на имя вашей переменной

# Настройка окружения
reg_setup = setup(data=df_reg, target='target', session_id=42, verbose=False)

# Сравнение моделей
reg_models = compare_models()
reg_results = pull()
print(reg_results)

# Вывод таблиц

In [ ]:
# === КЛАССИФИКАТОРЫ ===
classification_models = {
    "DecisionTree": optuna_clf,
    "Bagging": final_bag_clf,
    "GradientBoosting": rand_gb_clf_final,
    "Stacking": final_stack_random_clf,
    "CatBoost": final_cat_random_clf,
    "XGBoost": final_xgb_clf_random,
    "LightGBM": final_lgb_clf_grid,
}

clf_rows = []
for name, model in classification_models.items():
    y_train_pred = model.predict(X_train_clf)
    y_test_pred = model.predict(X_test_clf)

    clf_rows.append([
        name,
        accuracy_score(y_train_clf, y_train_pred),
        f1_score(y_train_clf, y_train_pred),
        precision_score(y_train_clf, y_train_pred),
        recall_score(y_train_clf, y_train_pred),
        accuracy_score(y_test_clf, y_test_pred),
        f1_score(y_test_clf, y_test_pred),
        precision_score(y_test_clf, y_test_pred),
        recall_score(y_test_clf, y_test_pred),
    ])

clf_df = pd.DataFrame(clf_rows, columns=[
    "Классификатор",
    "Train Accuracy", "Train F1", "Train Precision", "Train Recall",
    "Test Accuracy", "Test F1", "Test Precision", "Test Recall"
])

# === РЕГРЕССОРЫ ===
regression_models = {
    "DecisionTree": grid_reg,
    "Bagging": final_bag_reg_random,
    "GradientBoosting": optuna_gb_reg_final,
    "Stacking": final_stack_grid_reg,
    "CatBoost": final_cat_grid_reg,
    "XGBoost": final_xgb_reg_optuna,
    "LightGBM": final_lgb_reg_random,
}

reg_rows = []
for name, model in regression_models.items():
    y_train_pred = model.predict(X_train_reg)
    y_test_pred = model.predict(X_test_reg)

    reg_rows.append([
        name,
        r2_score(y_train_reg, y_train_pred),
        mean_absolute_error(y_train_reg, y_train_pred),
        mean_absolute_percentage_error(y_train_reg, y_train_pred),
        mean_squared_error(y_train_reg, y_train_pred),
        r2_score(y_test_reg, y_test_pred),
        mean_absolute_error(y_test_reg, y_test_pred),
        mean_absolute_percentage_error(y_test_reg, y_test_pred),
        mean_squared_error(y_test_reg, y_test_pred),
    ])

reg_df = pd.DataFrame(reg_rows, columns=[
    "Регрессор",
    "Train R2", "Train MAE", "Train MAPE", "Train MSE",
    "Test R2", "Test MAE", "Test MAPE", "Test MSE"
])

# === ВЫВОД ===
print("\n=== Таблица классификаторов ===")
print(clf_df.round(4))

print("\n=== Таблица регрессоров ===")
print(reg_df.round(4))

# Pycaret. Классификация

                                    Model  Accuracy    AUC  Recall  Prec.  \
dt               Decision Tree Classifier    1.0000 0.9999  0.9998 1.0000      
gbc          Gradient Boosting Classifier    0.9997 1.0000  0.9966 0.9996   
catboost              CatBoost Classifier    0.9987 1.0000  0.9935 0.9918   
xgboost         Extreme Gradient Boosting    0.9984 1.0000  0.9912 0.9903   
lightgbm  Light Gradient Boosting Machine    0.9984 1.0000  0.9906 0.9908   

# Мои вычисленные модели
                   Test Accuracy  Test F1  Test Precision  Test Recall  
Decision Tree Classifier  1.0000   0.9999          0.9999       0.9998  
GradientBoosting          1.0000   0.9999          0.9999       0.9998  
CatBoost                  0.9981   0.9890          0.9809       0.9972  
XGBoost                   0.9984   0.9911          0.9918       0.9904  
LightGBM                  0.9987   0.9923          0.9915       0.9932


Decision Tree показал одинаково идеальные результаты — Accuracy = 1.0000, F1 = 0.9999, Recall = 0.9998.

Gradient Boosting в моих расчётах дал лучшие метрики, чем в PyCaret: Accuracy = 1.0000 (vs. 0.9997), F1 = 0.9999, Recall = 0.9998 (vs. 0.9966).

CatBoost и XGBoost показали сопоставимые метрики, но у меня значения немного ниже (Accuracy у CatBoost 0.9981 против 0.9987, Recall у xgboost 0.9904 против 0.9912).

LightGBM у меня немного лучше по Accuracy (0.9987 vs. 0.9984), но разница минимальна.

3/4 метрик в моей реализации выше, чем у pycaret

# Pycaret. Регрессия
                                     RMSE       R2  RMSLE   MAPE  TT (Sec)  
CatBoost Classifier           114114.0777   0.9039 0.1594 0.1184    3.0650  
Light Gradient Boosting       125621.9194   0.8832 0.1690 0.1269    0.2040  
Extreme Gradient Boosting     127636.8909   0.8788 0.1716 0.1288    0.1930  
Gradient Boosting Classifier  137144.7756   0.8601 0.1878 0.1452    0.9200  
Decision Tree Classifier      197620.2824   0.7062 0.2529 0.1874    0.0800   

# Мои вычисленные модели
                          Test R2  Test MAE  Test MAPE         Test MSE  
Decision Tree Classifier  0.8249 85941.1076     0.1576 24256057467.3921  
GradientBoosting          0.8972 69303.3339     0.1324 14232421400.7882  
CatBoost Regressor        0.8898 70191.6245     0.1346 15266679657.9134  
XGBoost                   0.8759 73884.6695     0.1419 17194939176.5526  
Light Gradient Boosting   0.8935 66690.3424     0.1263 14752423427.8767  

Gradient Boosting показал лучшую обобщённую точность в моей реализации (R2 = 0.8972), а также одну из самых низких ошибок MAE и MAPE.

CatBoost имеет высшее R2 = 0.9039 в PyCaret.

LightGBM в моей реализации близок к CatBoost по R2 (0.8935), но имеет наименьший MAE = 66690.34 и самый низкий MAPE = 0.1263.

Decision Tree у меня лучше в сравнении с PyCaret (с 0.7062 против 0.8249). Достаточно большое различие

XGBoost метрики, полученные с помощью PyCaret, немного выше вычисленной вручную реализации (0.8788 против 0.8759)

In [ ]:
# Уменьшаем train для классификации до 0.5%
X_train_clf_small, _, y_train_clf_small, _ = train_test_split(
    X_train_clf, y_train_clf, train_size=0.005, stratify=y_train_clf, random_state=42
)

# KFold
kfold = KFold(n_splits=5, shuffle=True, random_state=42)

# ==== Классификаторы ====
clf_rows = []
for name, model in classification_models.items():
    # Hold-out
    y_pred = model.predict(X_test_clf)
    acc = accuracy_score(y_test_clf, y_pred)
    f1 = f1_score(y_test_clf, y_pred)
    prec = precision_score(y_test_clf, y_pred)
    rec = recall_score(y_test_clf, y_pred)

    # KFold на уменьшенной выборке
    acc_kf = cross_val_score(model, X_train_clf_small, y_train_clf_small, cv=kfold, scoring='accuracy', n_jobs=-1).mean()
    f1_kf = cross_val_score(model, X_train_clf_small, y_train_clf_small, cv=kfold, scoring='f1', n_jobs=-1).mean()
    prec_kf = cross_val_score(model, X_train_clf_small, y_train_clf_small, cv=kfold, scoring='precision', n_jobs=-1).mean()
    rec_kf = cross_val_score(model, X_train_clf_small, y_train_clf_small, cv=kfold, scoring='recall', n_jobs=-1).mean()

    clf_rows.append([
        name, acc, f1, prec, rec, acc_kf, f1_kf, prec_kf, rec_kf
    ])

clf_df = pd.DataFrame(clf_rows, columns=[
    "Классификатор",
    "Holdout_Accuracy", "Holdout_F1", "Holdout_Precision", "Holdout_Recall",
    "KFold_Accuracy", "KFold_F1", "KFold_Precision", "KFold_Recall"
])

# ==== Регрессоры ====
reg_rows = []
for name, model in regression_models.items():
    # Hold-out
    y_pred = model.predict(X_test_reg)
    mae = mean_absolute_error(y_test_reg, y_pred)
    mse = mean_squared_error(y_test_reg, y_pred)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_test_reg, y_pred)
    mape = mean_absolute_percentage_error(y_test_reg, y_pred)

    # KFold на полной выборке (с .iloc для поддержки DataFrame)
    mae_kf = -cross_val_score(model, X_train_reg, y_train_reg, cv=kfold, scoring='neg_mean_absolute_error', n_jobs=-1).mean()
    mse_kf = -cross_val_score(model, X_train_reg, y_train_reg, cv=kfold, scoring='neg_mean_squared_error', n_jobs=-1).mean()
    rmse_kf = np.sqrt(mse_kf)
    r2_kf = cross_val_score(model, X_train_reg, y_train_reg, cv=kfold, scoring='r2', n_jobs=-1).mean()

    mape_kf = np.mean([
        mean_absolute_percentage_error(
            y_train_reg.iloc[valid_idx],
            model.fit(X_train_reg.iloc[train_idx], y_train_reg.iloc[train_idx]).predict(X_train_reg.iloc[valid_idx])
        )
        for train_idx, valid_idx in kfold.split(X_train_reg)
    ])

    reg_rows.append([
        name, mae, mse, rmse, r2, mape, mae_kf, mse_kf, rmse_kf, r2_kf, mape_kf
    ])

reg_df = pd.DataFrame(reg_rows, columns=[
    "Регрессор",
    "Holdout_MAE", "Holdout_MSE", "Holdout_RMSE", "Holdout_R2", "Holdout_MAPE",
    "KFold_MAE", "KFold_MSE", "KFold_RMSE", "KFold_R2", "KFold_MAPE"
])

# === Вывод ===
print("\n=== Таблица классификаторов ===")
print(clf_df.round(4))

print("\n=== Таблица регрессоров ===")
print(reg_df.round(4))

# Вывод

***Классификация***

Все ансамблевые модели (Bagging, Stacking, GradientBoosting) и DecisionTree достигли идеального значения Accuracy = 1.0000, F1 = 0.9999, 
Precision = 0.9999.

Лучшие Recall (0.9999) наблюдаются у Bagging и Stacking.

Остальные модели также показывают отличные результаты, но уступают ансамблям.

Лучшая модель по классификации:
BaggingClassifier и StackingClassifier — абсолютно точные, стабильные по всем метрикам.



***Регрессия***

GradientBoosting дал лучший R2 = 0.8972 и наименьший MSE = 14.23e+9.

Stacking дал наименьший MAPE = 0.1258, что делает его лидером по относительной ошибке.

LightGBM дал наименьший MAE = 66690.34, что выгодно, если важна минимизация абсолютной ошибки.

DecisionTree — худший результат по всем метрикам.

Лучшая модель по регрессии:
GradientBoostingRegressor — наиболее сбалансированная модель с лучшими R2 и MSE.
StackingRegressor и LightGBM тоже сильны и могут быть выбраны в зависимости от приоритета (минимальный MAE или MAPE).


***Выбор между Holdout и K-Fold***

1. Классификация

Holdout

Все топ-4 модели (DecisionTree, Bagging, GradientBoosting, Stacking) показывают идеальные метрики:
Accuracy, F1, Precision, Recall

1.0000 / 0.9999 / 0.9999 / 0.9998–0.9999


K-Fold

У этих же моделей показатели хуже:

DecisionTree: F1 = 0.9639, Precision = 0.9500, Recall = 0.9790

Bagging: F1 = 0.9803, Precision = 0.9944, Recall = 0.9675

GradientBoosting: F1 = 0.9897, Precision = 0.9947, Recall = 0.9848

Stacking: F1 = 0.9801, Precision = 0.9972, Recall = 0.9646

**Заключение:**
Holdout даёт слишком оптимистичную оценку.
K-Fold показывает более реалистичную картину — даже у лучших моделей F1 < 0.99.

Лучше использовать: K-Fold — он выявляет возможное переобучение, особенно у простых моделей (DecisionTree).



2. Регрессия

Holdout

Лучшая модель по R²: GradientBoosting (R² = 0.8972, MAE = 69303, MAPE = 0.1324)

Stacking, LightGBM и Bagging близки, но немного уступают


K-Fold

Те же модели показывают хуже R²:
GradientBoosting: 0.8865
Stacking: 0.8768
LightGBM: 0.8907
Bagging: 0.8725

Разница в R² между Holdout и K-Fold — от 0.01 до 0.02, что не критично, но говорит о легкой переоценке качества в Holdout.

***Заключение***:
Обе стратегии дают схожий рейтинг моделей, но K-Fold снова честнее и стабильнее.

estimators = [ ('dt', DecisionTreeClassifier(random_state=42)), ('rf', RandomForestClassifier(random_state=42)) ] добавить третью(свм или линейна)